# Custom Directives

In this notebook, we'll explore how to create custom directives to encapsulate complex query patterns. Custom directives allow you to package reusable query logic that can be shared across multiple search applications.

## Introduction to Custom Directives

As your search applications grow in complexity, you'll often find yourself repeating the same query patterns. Custom directives solve this problem by allowing you to:

1. Encapsulate complex query logic in a reusable component
2. Hide implementation details behind a clean interface
3. Share query patterns across multiple applications
4. Maintain consistent search behavior

The toolkit provides the `CustomMatchDirective` class for creating custom directives.

## Setup

Let's import the necessary modules:

In [ ]:
import json
from elastictoolkit.queryutils.builder.custommatchdirective import CustomMatchDirective
from elastictoolkit.queryutils.builder.booldirective import AndDirective, OrDirective
from elastictoolkit.queryutils.builder.matchdirective import (
    ConstMatchDirective,
    TextMatchDirective,
    RangeMatchDirective,
    FieldExistsDirective
)
from elastictoolkit.queryutils.builder.directivevaluemapper import DirectiveValueMapper
from elastictoolkit.queryutils.types import FieldValue
from elastictoolkit.queryutils.consts import FieldMatchType, MatchMode
from elastictoolkit.queryutils.builder.directiveengine import DirectiveEngine

# Helper function to print queries as formatted JSON
def print_query(directive):
    query = directive.to_dsl()
    print(json.dumps(query.to_query(), indent=2))

## Creating a Basic Custom Directive

Let's start by creating a basic custom directive for a job search application. This directive will encapsulate the logic for matching candidates based on experience level and skills:

In [ ]:
# Define a value mapper for our job search application
class JobSearchValueMapper(DirectiveValueMapper):
    experience = FieldValue(
        fields=["experience"],
        values_list=["match_params.experience"]
    )
    skills = FieldValue(
        fields=["skills"],
        values_list=["*match_params.skills"]
    )
    location = FieldValue(
        fields=["location"],
        values_list=["match_params.location"]
    )

# Create a custom directive for candidate matching
class CandidateMatchDirective(CustomMatchDirective):
    allowed_engine_cls_name = "JobSearchEngine"  # This is required
    name = "candidate_match"  # Optional name for the directive
    
    def get_directive(self):
        # Return a boolean directive that combines multiple match conditions
        return AndDirective(
            # Match experience level
            experience=ConstMatchDirective(rule=FieldMatchType.ANY),
            # Match all required skills
            skills=ConstMatchDirective(rule=FieldMatchType.ALL)
        )

# Create a directive engine that uses our custom directive
class JobSearchEngine(DirectiveEngine):
    # Use our custom directive
    candidate_match = CandidateMatchDirective()
    # Add a location filter
    location = ConstMatchDirective(rule=FieldMatchType.ANY, nullable_value=True)
    
    class Config:
        value_mapper = JobSearchValueMapper()

# Create an engine instance with match parameters
engine = JobSearchEngine().set_match_params(
    {
        "experience": "senior",
        "skills": ["python", "elasticsearch", "aws"],
        "location": "remote"
    }
)

# Generate the query
print_query(engine)

In this example, we've created a custom directive `CandidateMatchDirective` that encapsulates the logic for matching candidates based on experience level and skills. The directive returns a boolean AND directive that combines multiple match conditions.

We've then used this custom directive in a `JobSearchEngine` to create a complete search query. The engine combines the custom directive with a location filter.

## Creating a More Complex Custom Directive

Let's create a more complex custom directive for a product search application. This directive will encapsulate the logic for matching products based on category, brand, and price range:

In [ ]:
# Define a value mapper for our product search application
class ProductSearchValueMapper(DirectiveValueMapper):
    category = FieldValue(
        fields=["category"],
        values_list=["match_params.category"]
    )
    brand = FieldValue(
        fields=["brand"],
        values_list=["*match_params.brands"]
    )
    price_range = FieldValue(
        fields=["price"],
        values_map={
            "gte": "match_params.min_price",
            "lte": "match_params.max_price"
        }
    )
    in_stock = FieldValue(
        fields=["in_stock"],
        values_list=[True]
    )
    search = FieldValue(
        fields=["name", "description"],
        values_list=["match_params.search_text"]
    )

# Create a custom directive for product filtering
class ProductFilterDirective(CustomMatchDirective):
    allowed_engine_cls_name = "ProductSearchEngine"
    name = "product_filter"
    
    def get_directive(self):
        # Return a boolean directive that combines multiple match conditions
        return AndDirective(
            # Always filter by in-stock status
            in_stock=ConstMatchDirective(rule=FieldMatchType.ANY),
            # Filter by category if provided
            category=ConstMatchDirective(rule=FieldMatchType.ANY, nullable_value=True),
            # Filter by price range if provided
            price_range=RangeMatchDirective(nullable_value=True),
            # Filter by brand if provided (any of the brands)
            brand=ConstMatchDirective(rule=FieldMatchType.ANY, nullable_value=True)
        )

# Create a directive engine that uses our custom directives
class ProductSearchEngine(DirectiveEngine):
    # Use our custom directives
    product_filter = ProductFilterDirective()
    search = TextMatchDirective(rule=FieldMatchType.ANY, nullable_value=True)
    
    class Config:
        value_mapper = ProductSearchValueMapper()

# Create an engine instance with match parameters
engine = ProductSearchEngine().set_match_params(
    {
        "category": "electronics",
        "min_price": 50,
        "max_price": 200,
        "brands": ["Apple", "Samsung"],
        "search_text": "wireless headphones"
    }
)

# Generate the query
print_query(engine)

In this example, we've created two custom directives:

1. `ProductFilterDirective`: Encapsulates the logic for filtering products based on in-stock status, category, price range, and brand
2. `ProductSearchTextDirective`: Encapsulates the logic for searching product name and description

We've then used these custom directives in a `ProductSearchEngine` to create a complete search query. The engine combines the filter directive with the search directive.

## Dynamic Custom Directives

Custom directives can also adapt their behavior based on match parameters. Let's create a dynamic custom directive for a real estate search application:

In [ ]:
# Define a value mapper for our real estate search application
class RealEstateValueMapper(DirectiveValueMapper):
    property_type = FieldValue(
        fields=["property_type"],
        values_list=["match_params.property_type"]
    )
    bedrooms = FieldValue(
        fields=["bedrooms"],
        values_map={
            "gte": "match_params.min_bedrooms",
            "lte": "match_params.max_bedrooms"
        }
    )
    price = FieldValue(
        fields=["price"],
        values_map={
            "gte": "match_params.min_price",
            "lte": "match_params.max_price"
        }
    )
    location = FieldValue(
        fields=["location"],
        values_list=["match_params.location"]
    )
    amenities = FieldValue(
        fields=["amenities"],
        values_list=["*match_params.amenities"]
    )

# Create a dynamic custom directive for property search
class PropertySearchDirective(CustomMatchDirective):
    allowed_engine_cls_name = "RealEstateEngine"
    
    def get_name(self):
        # Dynamic name based on property type
        property_type = self._match_params.get("property_type", "property")
        return f"{property_type}_search"
    
    def get_directive(self):
        # Create a base directive for all property searches
        base_directive = AndDirective(
            property_type=ConstMatchDirective(rule=FieldMatchType.ANY, nullable_value=True),
            bedrooms=RangeMatchDirective(nullable_value=True),
            price=RangeMatchDirective(nullable_value=True),
            location=ConstMatchDirective(rule=FieldMatchType.ANY, nullable_value=True)
        )
        
        # If amenities are specified, add them to the directive
        if "amenities" in self._match_params and self._match_params["amenities"]:
            # Add amenities directive
            amenities_directive = ConstMatchDirective(rule=FieldMatchType.ALL)
            base_directive.add_directive(amenities=amenities_directive)
        
        return base_directive

# Create a directive engine that uses our custom directive
class RealEstateEngine(DirectiveEngine):
    # Use our custom directive
    property_search = PropertySearchDirective()
    
    class Config:
        value_mapper = RealEstateValueMapper()

# Create an engine instance with match parameters
engine = RealEstateEngine().set_match_params(
    {
        "property_type": "apartment",
        "min_bedrooms": 2,
        "max_bedrooms": 3,
        "min_price": 1500,
        "max_price": 3000,
        "location": "downtown",
        "amenities": ["parking", "gym", "pool"]
    }
)

# Generate the query
print_query(engine)

In this example, we've created a dynamic custom directive `PropertySearchDirective` that adapts its behavior based on match parameters:

1. It generates a dynamic name based on the property type
2. It conditionally adds an amenities directive if amenities are specified

This approach allows you to create flexible, reusable query components that adapt to different search scenarios.

## Combining Multiple Custom Directives

You can also combine multiple custom directives to create complex search applications. Let's create a comprehensive e-commerce search engine that combines product filtering, search, and recommendation directives:

In [ ]:
# Define a value mapper for our e-commerce application
class ECommerceValueMapper(DirectiveValueMapper):
    category = FieldValue(
        fields=["category"],
        values_list=["match_params.category"]
    )
    price_range = FieldValue(
        fields=["price"],
        values_map={
            "gte": "match_params.min_price",
            "lte": "match_params.max_price"
        }
    )
    brand = FieldValue(
        fields=["brand"],
        values_list=["*match_params.brands"]
    )
    rating = FieldValue(
        fields=["rating"],
        values_map={
            "gte": "match_params.min_rating"
        }
    )
    in_stock = FieldValue(
        fields=["in_stock"],
        values_list=[True]
    )
    search = FieldValue(
        fields=["name", "description"],
        values_list=["match_params.search_text"]
    )
    user_preferences = FieldValue(
        fields=["tags"],
        values_list=["*match_params.user_preferences"]
    )

# Create a custom directive for basic product filtering
class ProductFilterDirective(CustomMatchDirective):
    allowed_engine_cls_name = "ECommerceEngine"
    name = "product_filter"
    
    def get_directive(self):
        return AndDirective(
            in_stock=ConstMatchDirective(rule=FieldMatchType.ANY),
            category=ConstMatchDirective(rule=FieldMatchType.ANY, nullable_value=True),
            price_range=RangeMatchDirective(nullable_value=True),
            brand=ConstMatchDirective(rule=FieldMatchType.ANY, nullable_value=True),
            rating=RangeMatchDirective(nullable_value=True)
        )

# Create a custom directive for product search
class ProductSearchDirective(CustomMatchDirective):
    allowed_engine_cls_name = "ECommerceEngine"
    name = "product_search"
    
    def get_directive(self):
        return TextMatchDirective(rule=FieldMatchType.ANY, nullable_value=True)

# Create a custom directive for personalized recommendations
class PersonalizedSearchRecommendationDirective(CustomMatchDirective):
    allowed_engine_cls_name = "ECommerceEngine"
    name = "personalized_search_recommendation"
    
    def get_directive(self):
        if not any((self._match_params.get("user_preferences"), self._match_params.get("search"))):
            return None

        # Since TextMatchDirective here is nullable - if `search` value is null then the query will not be added
        or_directive = OrDirective(search=TextMatchDirective(rule=FieldMatchType.ANY, nullable_value=True, name="search"))

        # Only apply if user preferences are provided
        if self._match_params.get("user_preferences"):
            # Match products that have at least one tag matching user preferences
            or_directive.add_directive(user_preferences=ConstMatchDirective(rule=FieldMatchType.ANY, name="user_preferences"))

        return or_directive

# Create a directive engine that combines all custom directives
class ECommerceEngine(DirectiveEngine):
    # Use our custom directives
    product_filter = ProductFilterDirective()
    personalized_search_recommendation = PersonalizedSearchRecommendationDirective()
    
    class Config:
        value_mapper = ECommerceValueMapper()

# Create an engine instance with match parameters
engine = ECommerceEngine().set_match_params(
    {
        "category": "electronics",
        "min_price": 50,
        "max_price": 200,
        "brands": ["Apple", "Samsung"],
        "min_rating": 4,
        "search_text": "wireless headphones",
        "user_preferences": ["bluetooth", "noise-cancelling", "wireless"]
    }
)

# Generate the query
print_query(engine)

In this example, we've created a comprehensive e-commerce search engine that combines three custom directives:

1. `ProductFilterDirective`: Handles basic product filtering (category, price, brand, rating, in-stock)
2. `ProductSearchDirective`: Handles text search across product name and description
3. `PersonalizedRecommendationDirective`: Adds personalized recommendations based on user preferences

This approach allows you to create modular, maintainable search applications by encapsulating different aspects of the search logic in separate custom directives.

## Summary

In this notebook, we've explored how to create custom directives to encapsulate complex query patterns. We've covered:

- Creating basic custom directives
- Building more complex custom directives
- Creating dynamic custom directives that adapt to match parameters
- Combining multiple custom directives to create comprehensive search applications

Custom directives provide a powerful way to encapsulate complex query logic in reusable components. They allow you to:

1. Hide implementation details behind a clean interface
2. Share query patterns across multiple applications
3. Create modular, maintainable search applications
4. Adapt query behavior based on runtime parameters

By using custom directives, you can create sophisticated search applications while keeping your code clean, modular, and maintainable.